# 📊 Análise Exploratória de Dados de da Rede Sonda 🌦️

## 📌 Introdução
Este notebook realiza uma análise exploratória dos dados meteorológicos coletados de diversas estações. O objetivo é entender a estrutura dos dados, avaliar sua qualidade e identificar padrões relevantes.

## 📂 Fonte dos Dados
- Arquivos CSV formatados armazenados no ftp
- Contêm medições de variáveis meteorológicas, solarimétricas e cameras.

## 🔍 Objetivos da Análise
1. **Carregar e explorar os dados**: verificar onde os dados estão armazenados, seu formato e estrutura.
2. **Dimensionamento e variáveis disponíveis**: entender o tamanho dos arquivos, número de registros e colunas.
3. **Análise temporal dos dados disponíveis**: identificar o período coberto e eventuais lacunas temporais.
4. **Visualização da distribuição espacial das estações**: verificar a abrangência geográfica das medições.
5. **Exploração inicial de distribuições**: histogramas e estatísticas básicas das variáveis.
6. **Análise de qualidade dos dados** *(última etapa)*: identificar valores ausentes, inconsistências e flags de qualidade.

### 1. Carregar e Explorar os Dados
Vamos começar listando o tamanho da base de dados que estão no diretório do ftp.

In [1]:
# Diretório onde os arquivos estão localizados
DIRETORIO = '../sonda/dados_formatados/'

In [2]:
# Exibe o tamanho de cada arquivo no diretório ordenado por tamanho de forma decrescente
!du -h --max-depth=1 {DIRETORIO} | sort -rh

11G	../sonda/dados_formatados/
1,2G	../sonda/dados_formatados/BRB
1011M	../sonda/dados_formatados/PTR
989M	../sonda/dados_formatados/FLN
959M	../sonda/dados_formatados/PMA
759M	../sonda/dados_formatados/JOI
722M	../sonda/dados_formatados/CPA
714M	../sonda/dados_formatados/SMS
686M	../sonda/dados_formatados/SLZ
613M	../sonda/dados_formatados/NAT
536M	../sonda/dados_formatados/CGR
464M	../sonda/dados_formatados/SBR
413M	../sonda/dados_formatados/TMA
365M	../sonda/dados_formatados/MCL
349M	../sonda/dados_formatados/ORN
300M	../sonda/dados_formatados/UBE
284M	../sonda/dados_formatados/BJL
175M	../sonda/dados_formatados/TLG
174M	../sonda/dados_formatados/CAI
171M	../sonda/dados_formatados/CTB
55M	../sonda/dados_formatados/CBA
196K	../sonda/dados_formatados/TRI
196K	../sonda/dados_formatados/SPK
196K	../sonda/dados_formatados/SCR
196K	../sonda/dados_formatados/RLM
196K	../sonda/dados_formatados/OPO
196K	../sonda/dados_formatados/MDS
196K	../sonda/dados_formatados/LEB
196K	../sonda/dados_form

Existem 3 tipos de dados:
- Dados Meteorológicos
- Dados Solarimétricos
- Dados Anemometricos

Abaixo vamos adicionar cada tipo de dado em uma lista para facilitar a análise.

In [3]:
import glob

# listar todos os dados Meteorológicos usando o glob só para o tipo de arquivo .csv
dados_metereologicos = glob.glob(DIRETORIO + "*/Meteorologicos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_metereologicos = [arquivo for arquivo in dados_metereologicos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Solarimétricos usando o glob só para o tipo de arquivo .csv
dados_solarimetricos = glob.glob(DIRETORIO + "*/Solarimetricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_solarimetricos = [arquivo for arquivo in dados_solarimetricos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Anemometricos usando o glob só para o tipo de arquivo .csv
dados_anemometricos = glob.glob(DIRETORIO + "*/Anemometricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_anemometricos = [arquivo for arquivo in dados_anemometricos if 'YYYY_MM' not in arquivo]

In [4]:
# Listar a quantidade de arquivos em cada categoria
print(f"Quantidade de arquivos Meteorologicos: {len(dados_metereologicos)}")
print(f"Quantidade de arquivos Solarimetricos: {len(dados_solarimetricos)}")
print(f"Quantidade de arquivos Anemometricos: {len(dados_anemometricos)}")

Quantidade de arquivos Meteorologicos: 1036
Quantidade de arquivos Solarimetricos: 1022
Quantidade de arquivos Anemometricos: 0


Utilizaremos a biblioteca duckdb para realizar a análise dos dados. DuckDB funciona como um banco de dados SQL, mas em memória, o que facilita a análise de grandes volumes de dados.

In [5]:
# Importar a biblioteca DuckDB para manipulação de dados
import duckdb
import os
import pandas as pd
import concurrent.futures


# Conectar ao banco de dados DuckDB e controla numero de threads
con = duckdb.connect(database=':memory:', read_only=False, config={'threads': 4})

# Remove qualquer arquivo temporário que possa existir
!rm -rf .tmp

In [6]:
# Função para criar a tabela no banco de dados
def criar_base(con, dados, base, arquivo):
    # Verifica se o arquivo já existe
    if os.path.exists(arquivo):
        con.execute(f"CREATE TABLE {base} AS SELECT * FROM read_parquet('{arquivo}')")
    else:
        try:
            query = f"""
            CREATE TABLE {base} AS 
            SELECT * FROM read_csv_auto('{dados[0]}',
                                        skip=2, 
                                        union_by_name=True, 
                                        all_varchar=True) 
            WHERE 1=0
            """
            con.execute(query)
            print(f"Tabela {base} criada com sucesso")
        except:
            print(f"Erro ao criar tabela {base}")

In [7]:
# Apontar o caminho das bases de dados
ARQV_METEOROLOGICO = '../sonda/dados_meteorologicos.parquet'
ARQV_SOLARIMETRICA = '../sonda/dados_solarimetricos.parquet'

In [8]:
# Criar a base de dados meteorológicos
criar_base(con, dados_metereologicos, 'base_meteorologica', ARQV_METEOROLOGICO)
# Criar a base de dados solarimétricos
criar_base(con, dados_solarimetricos, 'base_solarimetrica', ARQV_SOLARIMETRICA)

In [9]:
# Verifica o conteudo da tabela base_meteorologica
con.execute("SELECT * FROM base_meteorologica LIMIT 5").fetch_df()

,acronym,timestamp,year,day,min,tp_sfc,humid_sfc,press,rain,ws10_avg,ws10_std,wd10_avg,wd10_std
0,BJL,2014-06-01 00:00:00,2014,152,0,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
1,BJL,2014-06-01 00:10:00,2014,152,10,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
2,BJL,2014-06-01 00:20:00,2014,152,20,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
3,BJL,2014-06-01 00:30:00,2014,152,30,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
4,BJL,2014-06-01 00:40:00,2014,152,40,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0


In [10]:
# Verifica o conteudo da tabela base_solarimetrica
con.execute("SELECT * FROM base_solarimetrica LIMIT 5").fetch_df()

,acronym,timestamp,year,day,min,glo_avg,glo_std,glo_max,glo_min,dif_avg,...,dir_min,lw_avg,lw_std,lw_max,lw_min,temp_glo,temp_dir,temp_dif,temp_dome,temp_case
0,BJL,2014-06-01 00:00:00,2014,152,0,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
1,BJL,2014-06-01 00:01:00,2014,152,1,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
2,BJL,2014-06-01 00:02:00,2014,152,2,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
3,BJL,2014-06-01 00:03:00,2014,152,3,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
4,BJL,2014-06-01 00:04:00,2014,152,4,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Função para popular a base de dados
def inserir_dados(base, arquivo, con, periodico=True):
    # Lendo os novos dados do CSV, pulando a segunda linha mas mantendo o cabeçalho
    new_data = pd.read_csv(arquivo, skiprows=[1])
    # Formata os dados forçando para que sejam númericos, e caso não seja, substitui pelo valor original
    new_data = new_data.apply(pd.to_numeric, errors='coerce').fillna(new_data)
    # Converte a coluna de timestamp para datetime
    new_data['timestamp'] = pd.to_datetime(new_data['timestamp'], errors='coerce')
    # Pega o nome da estação do arquivo
    estacao = new_data['acronym'][0]
    # Pega o período de tempo do arquivo
    tempo = new_data['timestamp'].min(), new_data['timestamp'].max()
    # Coluna de variáveis
    variaveis = new_data.columns[5:]
    # Para cada variável
    for variavel in variaveis:
        if periodico:
            # Query periodica # Insere todos os dados de uma vez verificando se já existe
            query = f"SELECT COUNT(*) FROM {base} WHERE acronym = '{estacao}' AND timestamp >= '{tempo[0]}' AND timestamp <= '{tempo[1]}' AND {variavel} IS NOT NULL"
            if con.execute(query).fetchone()[0] > 0:
                print(f"Já existe dados para a estação {estacao}, no período de {tempo[0]} até {tempo[1]} para a variável {variavel}")
                continue
            else:
                try:
                    con.execute(f"INSERT INTO {base} VALUES ({', '.join([f'"{estacao}"'] + [f"'{row}'" for row in new_data[variavel]])})")
                except Exception as e:
                    print('Erro ao inserir dados do arquivo', arquivo, 'na base', base, 'para a variável', variavel)
                    print(e)
        else: # Não periodico # Insere um a um verificando se já existe
            for _, row in new_data[['timestamp', variavel]].iterrows():
                query = f"SELECT COUNT(*) FROM {base} WHERE acronym = '{estacao}' AND timestamp = '{row['timestamp']}' AND {variavel} IS NOT NULL"
                if con.execute(query).fetchone()[0] > 0:
                    print(f"Já existe dados para a estação {estacao}, no período de {row['timestamp']} para a variável {variavel}")
                    continue
                else:
                    try:
                        con.execute(f"INSERT INTO {base} VALUES ({', '.join([f'"{estacao}"'] + [f"'{row[col]}'" for col in new_data.columns[1:]])})")
                    except Exception as e:
                        print('Erro ao inserir dados do arquivo', arquivo, 'na base', base, 'para a variável', variavel)
                        print(e)

In [44]:
# def inserir_dados_paralelo(base, arquivos, con, periodico=True):
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         futures = [executor.submit(inserir_dados, base, arquivo, con, periodico) for arquivo in arquivos]
#         concurrent.futures.wait(futures)

def inserir_dados_paralelo(base, arquivos, con, periodico=True):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(inserir_dados, base, arquivo, con, periodico) for arquivo in arquivos]
        concurrent.futures.wait(futures)    
        
def inserir_dados_sequencial(base, arquivos, con, periodico=True):
    print(f"Inserindo dados na base {base} de forma sequencial")
    for arquivo in arquivos:
        inserir_dados(base, arquivo, con, periodico)

In [46]:
# Inserir os dados meteorológicos
inserir_dados_paralelo('base_meteorologica', dados_metereologicos[0:10], con, True)

In [43]:
inserir_dados_sequencial('base_meteorologica', dados_metereologicos[0:10], con, True)

Inserindo dados na base base_meteorologica de forma sequencial
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável tp_sfc
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável humid_sfc
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável press
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável rain
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável ws10_avg
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável ws10_std
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável wd10_avg
Já existe dados para a estação FLN, no período de 2019-03-01 00:00:00 até 2019-03-31 23:50:00 para a variável wd

In [ ]:
# Popular a base de dados solarimétricos
for arquivo in dados_solarimetricos:
    inserir_dados('base_solarimetrica', arquivo, con, periodico=True)

In [ ]:
# Função para criar a tabela no banco de dados
def criar_tabela(con, dados, base, saida):
    # Deleta tabela caso exista
    con.execute(f"DROP TABLE IF EXISTS {base}")

    # Verifica se o arquivo existe, caso exista leia o arquivo usando duckdb
    if  os.path.exists(saida):
        # Ler o arquivo parquet parquet usando duckdb
        con.execute(f"CREATE TABLE {base} AS SELECT * FROM read_parquet('{saida}')")
    else:
        # Criar a tabela temporária vazia com base no primeiro arquivo CSV
        query = f"""
        CREATE TABLE {base} AS 
        SELECT * FROM read_csv_auto('{dados[0]}', 
                                    skip=2, 
                                    union_by_name=True, 
                                    all_varchar=True) 
        WHERE 1=0
        """
        con.execute(query)  # Cria a tabela vazia com o esquema correto
        # Inserir os dados em lote sem carregar tudo na memória
        for arquivo in sorted(dados):
            print(f"Inserindo dados do arquivo: {arquivo}", end="\r", flush=True)
            try:
                query = f"""
                INSERT INTO {base} 
                SELECT * FROM read_csv_auto('{arquivo}', skip=2, union_by_name=True, all_varchar=True)
                """
                con.execute(query)
            except Exception as e:
                print(f"⚠️ Erro ao processar o arquivo: {arquivo}")
                print(f"   ➡️ Motivo: {e}")

        # Ler a primeira linha do arquivo CSV para obter os nomes das colunas
        header = pd.read_csv(dados[0], nrows=1)
        column_names = header.columns.tolist()  # Nomes das colunas

        # Renomear as colunas com base nos nomes do arquivo CSV
        for i, col in enumerate(column_names):
            # Formata o nome da coluna conforme a nomenclatura do DuckDB (column01, column02, ...)
            column_name = f"column{str(i).zfill(2)}"  # Preenche com zero à esquerda para 2 dígitos
            # Renomeia a coluna pelo nome correto
            con.execute(f"ALTER TABLE {base} RENAME COLUMN {column_name} TO {col}")

        # Set column timestamp as datetime
        con.execute(f"ALTER TABLE {base} ALTER COLUMN timestamp SET DATA TYPE TIMESTAMP")

        # Seta coluna 1 como STRING
        con.execute(f"ALTER TABLE {base} ALTER COLUMN {column_names[0]} SET DATA TYPE STRING")

        # Todas as demais colunas como FLOAT a partir da segunda coluna
        for i, col in enumerate(column_names[5:], start=5):
            # Formata o dado da coluna caso exista valores com , virgula, substitui por .
            con.execute(f"UPDATE {base} SET {col} = REPLACE({col}, ',', '.')")

            # Formata o dado da coluna caso exista valores com - virgula, substitui por .
            con.execute(f"UPDATE {base} SET {col} = REPLACE({col}, '-', '0')")

            try:
                con.execute(f"ALTER TABLE {base} ALTER COLUMN {col} SET DATA TYPE FLOAT")
            except Exception as e:
                print(f"⚠️ Erro ao processar a coluna: {col}")
                print(f"   ➡️ Motivo: {e}")

        # Seta colunas 2, 3 e 4 como INTEGER
        for i in column_names[2:5]:
            try:
                con.execute(f"ALTER TABLE {base} ALTER COLUMN {i} SET DATA TYPE INTEGER")
            except Exception as e:
                print(f"⚠️ Erro ao processar a coluna: {i}")
                print(f"   ➡️ Motivo: {e}")
        
        # Salvar os dados em Parquet na BASE
        con.execute(f"COPY {base} TO '{saida}' (FORMAT 'parquet')")

In [ ]:
# Apontar o caminho das bases de dados
BASE_METEOROLOGICA = '../sonda/dados_meteorologicos.parquet'
BASE_SOLARIMETRICA = '../sonda/dados_solarimetricos.parquet'

In [ ]:
# Criar a tabela meteorológica
tabela_meteorologica = 'base_meteorologica'
# criar_tabela(con, dados_metereologicos, tabela_meteorologica, BASE_METEOROLOGICA)

In [ ]:
# Criar a tabela solarimétrica
tabela_solarimetrica = 'base_solarimetrica'
# criar_tabela(con, dados_solarimetricos, tabela_solarimetrica, BASE_SOLARIMETRICA)

Com as bases de dados criadas e carregadas em memória, podemos começar a análise exploratória dos dados.

In [ ]:
# Exibir as primeiras linhas da tabela meteorological
con.execute(f"SELECT * FROM {tabela_meteorologica} LIMIT 5").fetch_df()

In [ ]:
# Exibir as primeiras linhas da tabela solarimetrica
tabela_solarimetrica = 'base_solarimetrica'
con.execute(f"SELECT * FROM {tabela_solarimetrica} LIMIT 5").fetch_df()

In [ ]:
# Pega apenas colunas de dados meteorológicos
colunas_meteoro = con.execute(f"SELECT * FROM {tabela_meteorologica} LIMIT 1").description
colunas_meteoro = [c[0] for c in colunas_meteoro[5:]]
print(f"Colunas de dados meteorológicos: {colunas_meteoro}")

# Pega apenas colunas de dados solarimétricos
colunas_solar = con.execute(f"SELECT * FROM {tabela_solarimetrica} LIMIT 1").description
colunas_solar = [c[0] for c in colunas_solar[5:]]
print(f"Colunas de dados solarimétricos: {colunas_solar}")

A primeira análise será a verificação temporal dos dados, para entender o período coberto e eventuais lacunas temporais.

In [ ]:
def verifica_temporal(con, base):
    # Faz um agrupamento por acronym e timestamp para verificar se os dados são temporais, fazendo a contagem de registros por dia
    query = f"""
    SELECT acronym, DATE_TRUNC('day', timestamp) AS data, COUNT(*) AS registros
    FROM {base}
    GROUP BY acronym, data
    ORDER BY data
    """
    return con.execute(query).fetch_df()

In [ ]:
# Verificar se os dados meteorológicos são temporais
verifica_temporal(con, tabela_meteorologica)

In [ ]:
def verificar_dados_invalidos(con, base, colunas): 
    query = f"""
    SELECT acronym, COUNT(*) AS total_dados,
    """
    # Para cada coluna em colunas_meteoro, criamos a parte do "dados_invalidos"
    for i, coluna in enumerate(colunas):
        query += f"""
        SUM(CASE WHEN \"{coluna}\" = 3333.0 THEN 1 ELSE 0 END) AS {coluna}_3333,
        SUM(CASE WHEN \"{coluna}\" = -5555.0 THEN 1 ELSE 0 END) AS {coluna}_minus_5555,
        """
    # Remover a última vírgula da consulta
    query = query.rstrip(",\n")
    # Adiciona a parte do FROM e GROUP BY
    query += f"""
    FROM \"{base}\"
    GROUP BY acronym
    """
    # Executar a query
    df = con.execute(query).fetch_df()
    return df

In [ ]:
verificar_dados_invalidos(con, tabela_meteorologica, colunas_meteoro)

In [ ]:
verificar_dados_invalidos(con, tabela_solarimetrica, colunas_solar)